
You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [1]:
import numpy as np
import pandas as pd


In [2]:
df=pd.read_csv (r"C:\Users\PMLS\Downloads\workspace (1) 2\workspace\bank_marketing.csv")

In [5]:
df.head(10)

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no
5,5,45,services,married,basic.9y,unknown,no,may,5,198,1,0,nonexistent,93.994,4.857,no
6,6,59,admin.,married,professional.course,no,no,may,3,139,1,0,nonexistent,93.994,4.857,no
7,7,41,blue-collar,married,unknown,unknown,no,may,12,217,1,0,nonexistent,93.994,4.857,no
8,8,24,technician,single,professional.course,no,yes,may,21,380,1,0,nonexistent,93.994,4.857,no
9,9,25,services,single,high.school,no,yes,may,5,50,1,0,nonexistent,93.994,4.857,no


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

# Performing operations for client CSV

In [9]:
df["job"].value_counts()

job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: count, dtype: int64

In [11]:
df["job"]=df["job"].str.replace('.', '_', regex=False)

## on education column

In [14]:
df["education"]=df["education"].str.replace('.', '_', regex=False)

In [16]:
df["education"]=df["education"].replace('unknown', np.NaN)

- now that we see that we have 1731 valueswe can replace it with most repeated value.

In [19]:
df["education"].isna().sum()

1731

In [21]:
repeated_value=df["education"].mode()[0]
print(repeated_value)

university_degree


- now that we see most repeated value is "university degree" we can replace the Nan values with university degree.

In [23]:
df["education"]=df["education"].fillna(repeated_value)

In [25]:
df["education"].isna().sum()


0

## credit default column
- First we will see how many unique valyes do we have.
- then we will replace any other values with Nan.
- then we will convert it into boolean


In [27]:
df["credit_default"].value_counts()

credit_default
no         32588
unknown     8597
yes            3
Name: count, dtype: int64

In [29]:
df['credit_default'] = df['credit_default'].apply(lambda x: 1 if x == 'yes' else 0)
df['credit_default'] = df['credit_default'].astype(bool)  


In [31]:
df["credit_default"].dtype

dtype('bool')

## Mortage Column
- Same as we did credit default. We will do with this one.
- first check any values that need to be replaced then convert into bool.

In [33]:
df['mortgage'].value_counts()

mortgage
yes        21576
no         18622
unknown      990
Name: count, dtype: int64

In [35]:
df['mortgage'] = df['mortgage'].apply(lambda x: 1 if x == 'yes' else 0)


In [39]:
df["mortgage"].dtype

dtype('int64')

In [41]:
df["mortgage"]=df["mortgage"].astype("bool")

## age column
- we need to find the total of isna values.
- After result, we can see that there are no null values in our age columns.

In [43]:

df["age"].isna().sum()


0

## client _id col
- We can see that we dont have any null values in this either.


In [45]:
df["client_id"].isna().sum()

0

## martial column

In [47]:
print(df["marital"].value_counts())

marital
married     24928
single      11568
divorced     4612
unknown        80
Name: count, dtype: int64


In [49]:
df["marital"]=df["marital"].replace("unknown", np.NaN)

In [51]:
df["marital"].isna().sum()

80

In [53]:
mode_value = df['marital'].mode()[0]
print(mode_value)

married


- we will fill the missing value with mode ( the most repeated value that is mmarried.)

In [55]:
df["marital"]=df["marital"].fillna(mode_value)

In [57]:
df["marital"].isna().sum() 

0

- now we can see that there are no null values.

# Making a csv of client

In [59]:
client_csv=df[["client_id", "age", "job", "marital", "education", "credit_default","mortgage"]]

In [61]:
client_csv.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,False,False
1,1,57,services,married,high_school,False,False
2,2,37,services,married,high_school,False,True
3,3,40,admin_,married,basic_6y,False,False
4,4,56,services,married,high_school,False,False


In [63]:
client_csv.isna().sum()

client_id         0
age               0
job               0
marital           0
education         0
credit_default    0
mortgage          0
dtype: int64

- finally saving it as client_csv


In [65]:
client_csv.to_csv("client.csv", index=False)

# Campaign csv data 

- since client id had no N/A so we will move on to the next col

## Number Contacts
- there are no null values in numner contacts.

In [70]:
df["number_contacts"].isna().sum()

0

## Contact_duration

- No null values, so moving on to next column for csv data

In [76]:
df["contact_duration"].isna().sum()

0

## previous_campaign_contacts column

In [79]:
df["previous_campaign_contacts"].isna().sum()

0

## campaign outcome column


In [83]:
df["campaign_outcome"].isna().sum()

0

In [85]:
df["campaign_outcome"].value_counts()

campaign_outcome
no     36548
yes     4640
Name: count, dtype: int64

- since there are no other values than yes and no so we cann directly change it to boolean

In [90]:
df['campaign_outcome'] = df['campaign_outcome'].apply(lambda x: 1 if x == 'yes' else 0)

In [92]:
print(df['campaign_outcome'] )

0        0
1        0
2        0
3        0
4        0
        ..
41183    1
41184    0
41185    0
41186    1
41187    0
Name: campaign_outcome, Length: 41188, dtype: int64


In [94]:
df["campaign_outcome"]=df["campaign_outcome"].astype("bool")

In [96]:
df["campaign_outcome"].dtype

dtype('bool')

## previous outcome column

In [99]:
df["previous_outcome"].isna().sum()

0

In [101]:
df["previous_outcome"].value_counts()

previous_outcome
nonexistent    35563
failure         4252
success         1373
Name: count, dtype: int64

- Now we can change the non- existent and failure to 0 and success to 1

In [104]:
df['previous_outcome'] = df['previous_outcome'].apply(lambda x: 1 if x == 'success' else 0)


In [106]:
df["previous_outcome"]=df["previous_outcome"].astype("bool")

In [108]:
df["previous_outcome"].dtype

dtype('bool')

## Last Contact Hour

- since our month column was strimg so we need to map it to number 

In [114]:
df["month"]

0        may
1        may
2        may
3        may
4        may
        ... 
41183    nov
41184    nov
41185    nov
41186    nov
41187    nov
Name: month, Length: 41188, dtype: object

In [118]:

month_map = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}

def convert_month(m):  # it will do conversion
    if isinstance(m, str):
        return month_map.get(m.lower())
    return m



In [120]:
df['month_num'] = df['month'].apply(convert_month) # now we are making a new column that contains the numbered month imstead of string like  may , june

df['year'] = 2022 # making a new  col that has a single value for all enteries 

# Creating the new date column combining year, month, day
df['last_contact_date'] = pd.to_datetime({
    'year': df['year'],
    'month': df['month_num'],
    'day': df['day']
}, errors='coerce')

#now we are formatting tehe date as "YYYY-MM-DD"
df['last_contact_date'] = df['last_contact_date'].dt.strftime('%Y-%m-%d')

In [122]:
print(df["last_contact_date"].head(10))

0    2022-05-13
1    2022-05-19
2    2022-05-23
3    2022-05-27
4    2022-05-03
5    2022-05-05
6    2022-05-03
7    2022-05-12
8    2022-05-21
9    2022-05-05
Name: last_contact_date, dtype: object


## Making a CSV

In [125]:
campaign_csv=df[["client_id", "number_contacts", "contact_duration", "previous_campaign_contacts", "previous_outcome", "campaign_outcome","last_contact_date"]]

In [127]:
campaign_csv.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,1,261,0,False,False,2022-05-13
1,1,1,149,0,False,False,2022-05-19
2,2,1,226,0,False,False,2022-05-23
3,3,1,151,0,False,False,2022-05-27
4,4,1,307,0,False,False,2022-05-03


In [129]:
campaign_csv.to_csv("campaign.csv", index=False)

## Economics csv

- Since client id has no null values so we can move onto next column for this csv

In [135]:
df["cons_price_idx"].isnull().sum()

0

In [137]:
df["euribor_three_months"].isnull().sum()

0

## Making CSV
so we will make a csv now.

In [143]:
economics_csv=df[["client_id","cons_price_idx","euribor_three_months"]]

In [145]:
economics_csv.head(6)

,client_id,cons_price_idx,euribor_three_months
0,0,93.994,4.857
1,1,93.994,4.857
2,2,93.994,4.857
3,3,93.994,4.857
4,4,93.994,4.857
5,5,93.994,4.857


In [147]:
economics_csv.to_csv("economics.csv", index=False)

#  Final CSVs

In [150]:
df_client=pd.read_csv("client.csv")
df_campaign=pd.read_csv("campaign.csv")

df_economics=pd.read_csv("economics.csv")

In [152]:
df_client.head(10)

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,False,False
1,1,57,services,married,high_school,False,False
2,2,37,services,married,high_school,False,True
3,3,40,admin_,married,basic_6y,False,False
4,4,56,services,married,high_school,False,False
5,5,45,services,married,basic_9y,False,False
6,6,59,admin_,married,professional_course,False,False
7,7,41,blue-collar,married,university_degree,False,False
8,8,24,technician,single,professional_course,False,True
9,9,25,services,single,high_school,False,True


In [154]:
df_campaign.head(10)

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,1,261,0,False,False,2022-05-13
1,1,1,149,0,False,False,2022-05-19
2,2,1,226,0,False,False,2022-05-23
3,3,1,151,0,False,False,2022-05-27
4,4,1,307,0,False,False,2022-05-03
5,5,1,198,0,False,False,2022-05-05
6,6,1,139,0,False,False,2022-05-03
7,7,1,217,0,False,False,2022-05-12
8,8,1,380,0,False,False,2022-05-21
9,9,1,50,0,False,False,2022-05-05


In [156]:
df_economics.head(10)

,client_id,cons_price_idx,euribor_three_months
0,0,93.994,4.857
1,1,93.994,4.857
2,2,93.994,4.857
3,3,93.994,4.857
4,4,93.994,4.857
5,5,93.994,4.857
6,6,93.994,4.857
7,7,93.994,4.857
8,8,93.994,4.857
9,9,93.994,4.857


### Thus, we have successfully converted our one csv file into 3 with proper cleaning.